In [ ]:
#@title 환경설정

from pathlib import Path

#@markdown ---
#@markdown ##***워크스페이스***
OPTIONS = {}

#@markdown Google Drive 사용 (용량에 주의)
USE_GOOGLE_DRIVE = False  #@param {type:"boolean"}
DRIVE_WORKSPACE = "/content/drive/MyDrive/ComfyUI" #@param {type:"string"}
#@markdown ComfyUI 업데이트하기 
UPDATE_COMFY_UI = False  #@param {type:"boolean"}
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
LODE_MODE = "aria2c --console-log-level=error -c -x 16 -s 16 -k 1M"

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')
    WORKSPACE = DRIVE_WORKSPACE
else:
   WORKSPACE = "/content/ComfyUI"


# 패키지 설치
!apt -y install -qq aria2
!{LODE_MODE} https://github.com/koya-jp/AA-google-colab-kohya/raw/master/libtcmalloc_minimal.so.4 -d /content -o libtcmalloc_minimal.so.4
%env LD_PRELOAD=/content/libtcmalloc_minimal.so.4
!pip install -q xformers==0.0.20 triton==2.0.0 -U
!pip install -q mediapipe==0.9.1.0 addict yapf fvcore omegaconf

# WORKSPACE가 존재하면 처리를 건너뜀 
![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI $WORKSPACE
%cd $WORKSPACE

# Git 업데이트 
if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =- && git pull

# 종속성 설치 
!echo -= Install requirements.txt =-
!pip install -q -r requirements.txt
!{LODE_MODE} https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -d /content -o cloudflared-linux-amd64
!chmod 777 /content/cloudflared-linux-amd64


#@markdown ⭐Google Drive (ComfyUI/output)에 이미지만 저장하는 설정
SAVE_DRIVE = True #@param {type:"boolean"}
if SAVE_DRIVE:
    # Google Drive에 연결 
    from google.colab import drive
    drive.mount('/content/drive')
    # 이미지 저장용 폴더 생성 ※ComdyUI 업데이트에 따라 사용할 수 없게 될 수 있음 
    !mkdir -p /content/drive/MyDrive/ComfyUI/output
    # 저장위치 재작성 
    !sed -i -e "s/output_directory = os.path.join(os.path.dirname(os.path.realpath(__file__)), \"output\")/output_directory = \"\/content\/drive\/MyDrive\/ComfyUI\/output\"/" /content/ComfyUI/folder_paths.py


#@markdown ---

# 모델/체크포인트/VAE 또는 커스텀 ComfyUI 노드 및 몇가지 다운로드하기 (필요한 것에 관련 명령어 주석 제거)


# custom node 설치 
CUSTOM_NODE_DIR = WORKSPACE + "/custom_nodes"
!git clone -b v1.5 https://github.com/camenduru/comfy_controlnet_preprocessors {CUSTOM_NODE_DIR}/comfy_controlnet_preprocessors
!git clone https://github.com/ltdrdata/ComfyUI-Manager {CUSTOM_NODE_DIR}/ComfyUI-Manager
# !git clone https://github.com/pythongosssss/ComfyUI-Custom-Scripts {CUSTOM_NODE_DIR}/ComfyUI-Custom-Scripts

%cd {CUSTOM_NODE_DIR}/comfy_controlnet_preprocessors
!python install.py --no_download_ckpts


#@markdown ##***Checkpoints***
# workflow examples: https://comfyanonymous.github.io/ComfyUI_examples/sdxl/
%cd $WORKSPACE

# base
#@markdown Base모델 다운로드 
INSTALL_BASE_MODEL = True  #@param {type:"boolean"}
BASE_URL = "https://civitai.com/api/download/models/141958" #@param {type:"string"}
BASE_NEME = "BreakDomainXL.safetensors"  #@param {type:"string"}
if INSTALL_BASE_MODEL:
    !{LODE_MODE} {BASE_URL} -d {WORKSPACE}/models/checkpoints -o {BASE_NEME}

# refiner
#@markdown Refiner모델 다운로드 
INSTALL_REFINER = False  #@param {type:"boolean"}
REFINER_URL = "https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors" #@param {type:"string"}
REFINER_NEME = "sd_xl_refiner_1.0.safetensors"  #@param {type:"string"}
if INSTALL_REFINER:
    !{LODE_MODE} {REFINER_URL} -d {WORKSPACE}/models/checkpoints -o {REFINER_NEME}

# vae
#@markdown VAE 다운로드 
INSTALL_VAE = False  #@param {type:"boolean"}
VAE_URL = "https://huggingface.co/ckpt/sdxl_vae/resolve/main/sdxl_vae.safetensors"
if INSTALL_VAE:
    !{LODE_MODE} {VAE_URL} -d {WORKSPACE}/models/vae -o sdxl_vae.vae.safetensors

# embedding
#@markdown embedding 다운로드 
INSTALL_EMBEDDING = False  #@param {type:"boolean"}
EMBEDDING_URL = "https://huggingface.co/okozk/ComfyUI/resolve/main/unaestheticXLv13.safetensors" #@param {type:"string"}
EMBEDDING_NEME = "unaestheticXLv13.safetensors"  #@param {type:"string"}
if INSTALL_EMBEDDING:
    !{LODE_MODE}  -d {WORKSPACE}/models/embeddings -o {EMBEDDING_NEME}

# lora
#@markdown lora 다운로드 
INSTALL_LORA = False  #@param {type:"boolean"}
LORA_URL = "https://civitai.com/api/download/models/137124" #@param {type:"string"}
LORA_NEME = "DreamART-Style.safetensors" #@param {type:"string"}
LORA_URL2 = "https://civitai.com/api/download/models/135867" #@param {type:"string"}
LORA_NEME2 = "detail-tweaker-xl.safetensors" #@param {type:"string"}
if INSTALL_LORA:
    !{LODE_MODE} {LORA_URL} -d {WORKSPACE}/models/loras -o {LORA_NEME}
    !{LODE_MODE} {LORA_UR2L} -d {WORKSPACE}/models/loras -o {LORA_NEME2}

# controlnet
#@markdown controlnet모델 다운로드 
INSTALL_CONTROLNET_MODEL = False  #@param {type:"boolean"}
if INSTALL_CONTROLNET_MODEL:
    !{LODE_MODE} https://huggingface.co/thibaud/controlnet-openpose-sdxl-1.0/resolve/main/OpenPoseXL2.safetensors -d {WORKSPACE}/models/controlnet -o OpenPoseXL2.safetensors
    !{LODE_MODE} https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-canny-rank256.safetensors -d {WORKSPACE}/models/controlnet -o control-lora-canny-rank256.safetensors
    !{LODE_MODE} https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-depth-rank256.safetensors -d {WORKSPACE}/models/controlnet -o control-lora-depth-rank256.safetensors
    !{LODE_MODE} https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-recolor-rank256.safetensors -d {WORKSPACE}/models/controlnet -o control-lora-recolor-rank256.safetensors
    !{LODE_MODE} https://huggingface.co/ckpt/controlnet-sdxl-1.0/resolve/main/control-lora-sketch-rank256.safetensors -d {WORKSPACE}/models/controlnet -o control-lora-sketch-rank256.safetensors

# upscaler
#@markdown upscaler모델 다운로드 
INSTALL_UPSCALER = False  #@param {type:"boolean"}
if INSTALL_UPSCALER:
    !{LODE_MODE} https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth -d {WORKSPACE}/models/upscale_models -o RealESRGAN_x2plus.pth

# 확인 
!echo -= 다운로드한 파일 확인 =-
%cd {WORKSPACE}/models/
!ls -la checkpoints vae upscale_models embeddings loras controlnet

!echo -= Done! =-
#@markdown ---

# referred to:https://github.com/comfyanonymous/ComfyUI,https://github.com/camenduru/gperftools

### Run ComfyUI




In [ ]:
%cd $WORKSPACE
!ls -la {WORKSPACE}/models/embeddings # embedding 적용시 파일명 필요 


import atexit, requests, subprocess, time, re, os
from random import randint
from threading import Timer
from queue import Queue


def cloudflared(port, metrics_port, output_queue):
    atexit.register(lambda p: p.terminate(), subprocess.Popen(['/content/cloudflared-linux-amd64', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
    attempts, tunnel_url = 0, None
    while attempts < 10 and not tunnel_url:
        attempts += 1
        time.sleep(3)
        try:
            tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
        except:
            pass
    if not tunnel_url:
        raise Exception("Can't connect to Cloudflare Edge")
    output_queue.put(tunnel_url)

output_queue, metrics_port = Queue(), randint(8100, 9000)
thread = Timer(2, cloudflared, args=(8188, metrics_port, output_queue))
thread.start()
thread.join()
tunnel_url = output_queue.get()
os.environ['webui_url'] = tunnel_url
print(tunnel_url)

# 실행 
!python main.py --dont-print-server

In [ ]:
# @title （예비）드라이브에 연결되지 않는 경우 압축된 ZIP파일을 드라이브에 다운로드 
from datetime import datetime
from google.colab import drive

drive.mount('/content/drive')

now = datetime.now()
date_time = now.strftime("%Y%m%d-%H%M%S")
!zip -r /content/output.zip /content/ComfyUI/output
!cp /content/output.zip /content/drive/MyDrive/output-{date_time}.zip